This notebook gets all the events from the `meetbot@ideo.com` calendar so that we can quantify impact and connections !!

In [1]:
import datetime
import dateutil.parser
import httplib2
import os
import pandas as pd
import traces

from apiclient import discovery
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage

import settings

In [2]:
def get_credentials(self=None):
    home_dir = os.path.expanduser('~')
    credential_dir = os.path.join(home_dir, '.credentials')
    if not os.path.exists(credential_dir):
        os.makedirs(credential_dir)
    credential_path = os.path.join(credential_dir,
                                       'calendar-python-quickstart.json')

    store = Storage(credential_path)
    credentials = store.get()
    if not credentials or credentials.invalid:
        flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
        flow.user_agent = APPLICATION_NAME
        if flags:
            credentials = tools.run_flow(flow, store, flags)
        else:  # Needed only for compatibility with Python 2.6                                                
            credentials = tools.run(flow, store)
        print('Storing credentials to ' + credential_path)
    return credentials

In [3]:
credentials = get_credentials()
http = credentials.authorize(httplib2.Http())
service = discovery.build('calendar', 'v3', http=http)

now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time                              

page_token = None
while True:
    events = service.events().list(calendarId='primary', 
                                   pageToken=page_token,
                                   timeMax=now # only get events that have happened already
                                  ).execute()
    page_token = events.get('nextPageToken')
    if not page_token:
        break

In [4]:
# get total number of attendees (who did not DECLINE the invitation)                                      
all_attendees = []
event_titles = []
events_with_emails = []

for event in events['items']:
    if len(event['attendees']) > 1:
        event_titles.append(event['summary'])
        for attendee in event['attendees']:
            if attendee['responseStatus'] is not 'declined':
                all_attendees.append(attendee['email'])
                events_with_emails.append((event['id'], attendee['email']))
print(len(set(all_attendees)), len(event_titles))

166 171


In [10]:
# Get counts of different types of events 
from collections import Counter
print(Counter(event_titles))

Counter({"Meet n' Three!": 134, 'D4AI Global Update Call!': 23, 'DR Dates!': 13, "Meet n' Four!": 1})


In [6]:
# 
d_index = pd.MultiIndex.from_tuples(events_with_emails, names=('event_id','email'))
events_df = pd.DataFrame(index=d_index)

In [15]:
for event in events['items']:
    print(event['start'])

{'dateTime': '2018-05-16T12:00:00-05:00'}
{'dateTime': '2018-05-24T12:00:00-05:00'}
{'dateTime': '2018-05-11T12:00:00-05:00'}
{'dateTime': '2018-05-10T12:00:00-05:00'}
{'dateTime': '2018-05-08T12:00:00-05:00'}
{'dateTime': '2018-05-08T12:00:00-05:00'}
{'dateTime': '2018-05-01T12:00:00-05:00'}
{'dateTime': '2018-05-15T12:00:00-05:00'}
{'dateTime': '2018-05-23T12:00:00-05:00'}
{'dateTime': '2018-05-08T12:00:00-05:00'}
{'dateTime': '2018-05-03T12:00:00-05:00'}
{'dateTime': '2018-05-04T12:00:00-05:00'}
{'dateTime': '2018-05-11T12:00:00-05:00'}
{'dateTime': '2018-05-11T12:00:00-05:00'}
{'dateTime': '2018-05-11T13:00:00-05:00'}
{'dateTime': '2018-05-11T09:00:00-05:00'}
{'dateTime': '2018-05-11T12:00:00-05:00'}
{'dateTime': '2018-05-10T12:00:00-05:00'}
{'dateTime': '2018-05-11T12:00:00-05:00'}
{'dateTime': '2018-05-17T12:00:00-05:00'}
{'dateTime': '2018-05-16T12:00:00-05:00'}
{'dateTime': '2018-05-17T12:00:00-05:00'}
{'dateTime': '2018-05-15T12:00:00-05:00'}
{'dateTime': '2018-05-15T12:00:00-

In [8]:
file_path = settings.DATA_DIRECTORY + 'directory_data.csv'
directory_data = pd.read_csv(file_path,  
                             usecols=['discipline','email'],
                             names=['discipline','email'],
                            skiprows=1) 
directory_data.set_index('email', inplace=True)

In [9]:
events_df.join(directory_data).head()

discipline
event_id                   email                                   
2bc6dekamdftiahr2m3tr8balc jmassa@ideo.com          Business Design
                           lpatterson@ideo.com               Talent
                           mgilmansmith@ideo.com         Technology
v3vku45og0s8o874o30pjkv800 ctorres@ideo.com       Industrial Design
                           msantana@ideo.com             Accounting